In [8]:
import gymnasium as gym
import numpy as np

In [25]:
env = gym.make("FrozenLake-v1",is_slippery=True)
policy = np.ones((16,4))/4

for episode in range(10000):
    state,_ = env.reset()
    states,actions,rewards = [],[],[]
    done = False
    while not done:
        action = np.random.choice(4,p=policy[state])
        next_state,reward,done,_,_ = env.step(action)
        states.append(state)
        rewards.append(reward)
        actions.append(action)
        state=next_state

    G=0
    returns=[]
    for r in reversed(rewards):
        G = r+0.99*G
        returns.insert(0,G)

    if returns:
        baseline = np.mean(returns)
        for s,a,Gt in zip(states,actions,returns):
            policy[s,a] += 0.1 * (Gt-baseline)
            policy[s] = np.maximum(policy[s],0.01)
            policy[s] /= np.sum(policy[s])

success=0
for _ in range(100):
    state,_ = env.reset()
    done = False
    while not done:
        action = np.argmax(policy[s])
        state,reward,done,_,_ = env.step(action)
        success += reward

print(f"Success: {success/100}")
arrows=['<','v','>','^']
for s in range(16):
    best_a = np.argmax(policy[s])
    print(f"State: {s} {arrows[best_a]} probs: {np.round(policy[s],3)}")

Success: 0.06
State: 0 v probs: [0.219 0.278 0.234 0.269]
State: 1 v probs: [0.251 0.292 0.262 0.195]
State: 2 > probs: [0.247 0.236 0.307 0.209]
State: 3 v probs: [0.242 0.261 0.241 0.256]
State: 4 ^ probs: [0.255 0.258 0.228 0.259]
State: 5 < probs: [0.25 0.25 0.25 0.25]
State: 6 v probs: [0.263 0.302 0.231 0.203]
State: 7 < probs: [0.25 0.25 0.25 0.25]
State: 8 v probs: [0.205 0.294 0.267 0.235]
State: 9 > probs: [0.228 0.265 0.282 0.225]
State: 10 v probs: [0.275 0.293 0.29  0.142]
State: 11 < probs: [0.25 0.25 0.25 0.25]
State: 12 < probs: [0.25 0.25 0.25 0.25]
State: 13 > probs: [0.196 0.241 0.283 0.279]
State: 14 > probs: [0.081 0.281 0.377 0.261]
State: 15 < probs: [0.25 0.25 0.25 0.25]
